In [15]:
import mlflow
import cloudpickle
import os

# Define the path to save the model
model_path = "./model/model.pkl"

# Create the directory if it does not exist
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# Set the MLflow tracking URI
mlflow.set_tracking_uri("http://seito.lavbic.net:5000")

# Load the model from MLflow registry
model_uri = "models:/Race prediction@production"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Create a simple wrapper class that can be pickled
class ModelWrapper:
	def __init__(self, mlflow_model):
		self.mlflow_model = mlflow_model
		
	def predict(self, X):
		return self.mlflow_model.predict(X)

# Create wrapper instance
wrapper = ModelWrapper(loaded_model)

# Save the wrapper using cloudpickle which handles more complex objects
print(f"Saving model wrapper to '{model_path}'...")
with open(model_path, 'wb') as f:
	cloudpickle.dump(wrapper, f)
print("Model wrapper saved successfully.")

Saving model wrapper to './model/model.pkl'...
Model wrapper saved successfully.


In [6]:
import requests
import json

# URL of the Flask server (adjust the port if necessary)
url = "http://seito.lavbic.net:15000/predict"

# Index of the race data you want to predict
payload = {
    "index": 0  # Replace with the actual index you want to use
}

# Send the POST request
try:
    response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(payload), timeout=60)
    
    # Check if the response is successful
    if response.status_code == 200:
        result = response.json()
        print("Prediction result:", result['prediction'])
    else:
        print("Error:", response.text)
except requests.exceptions.RequestException as e:
    print("Request failed:", e)


Prediction result: [{'image_url': '../common/images/Pepijn  Reinderink.jpg', 'name': 'Pepijn  Reinderink', 'prediction': 0.6224539279937744}, {'image_url': '../common/images/Anders  Foldager.jpg', 'name': 'Anders  Foldager', 'prediction': 0.5724022388458252}, {'image_url': '../common/images/Antoine  Huby.jpg', 'name': 'Antoine  Huby', 'prediction': 0.5442665219306946}, {'image_url': '../common/images/Lars  Craps.jpg', 'name': 'Lars  Craps', 'prediction': 0.5098865032196045}, {'image_url': '../common/images/Brandon  McNulty.jpg', 'name': 'Brandon  McNulty', 'prediction': 0.5013190507888794}, {'image_url': '../common/images/Dylan  Vandenstorme.jpg', 'name': 'Dylan  Vandenstorme', 'prediction': 0.49420881271362305}, {'image_url': '../common/images/Gil  Gelders.jpg', 'name': 'Gil  Gelders', 'prediction': 0.464415580034256}, {'image_url': '../common/images/William Junior  Lecerf.jpg', 'name': 'William Junior  Lecerf', 'prediction': 0.4345358908176422}, {'image_url': '../common/images/Carlos